In [24]:
import numpy    as np
import torch.nn as nn
import torch
import json

from torch_geometric.data import Batch
from libraries.model      import nGCNN, eGCNN, denoise, get_random_graph, add_features_to_graph
from libraries.dataset    import revert_standardize_dataset
from libraries.graph      import POSCAR_graph_encoding

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# From random noise, we generate completely new materials
# A target property can be seeked with this approach

In [7]:
# Define folder in which all data will be stored
target_folder    = 'models/Loaded_QM9_gap-sphere-images/GM_v9'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

# Number of graphs to predict
N_predictions = 10

# Define target to be generated
target_tensor = torch.tensor(1, dtype=torch.int, device=device)

# Load model data

In [8]:
# Read the file in JSON format to a dictionary
with open(f'{target_folder}/model_parameters.json', 'r') as json_file:
    numpy_dict = json.load(json_file)

# Convert torch tensors to numpy arrays
model_parameters = {}
for key, value in numpy_dict.items():
    try:
        model_parameters[key] = torch.tensor(value, device=device)
    except:
        model_parameters[key] = value

# Number of diffusing and denoising steps
n_t_steps = model_parameters['n_t_steps']

# Decay of parameter alpha
noise_contribution = model_parameters['noise_contribution']
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = model_parameters['dropout_node']
dropout_edge = model_parameters['dropout_edge']

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [9]:
dataset_name                = f'{target_folder}/dataset.pt'
labels_name                 = f'{target_folder}/standardized_labels.pt'
dataset_name_std            = f'{target_folder}/standardized_dataset.pt'
dataset_parameters_name_std = f'{target_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions

# Load the standardized dataset
dataset = torch.load(dataset_name_std)

# Read the file in JSON format to a dictionary
with open(dataset_parameters_name_std, 'r') as json_file:
    numpy_dict = json.load(json_file)

# Convert torch tensors to numpy arrays
dataset_parameters = {}
for key, value in numpy_dict.items():
    try:
        dataset_parameters[key] = torch.tensor(value, device=device)
    except:
        dataset_parameters[key] = value

/tmp/ipykernel_362992/933877349.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(dataset_name_std)


In [12]:
# Normalize target_tensor accordingly
target_tensor = (target_tensor - dataset_parameters['target_mean']) * dataset_parameters['scale'] / dataset_parameters['target_std']

In [13]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

(17.983739852905273, 2.9542582035064697)

# Loading the model

In [21]:
# Determine number of node-level features in dataset, considering the t_step information
n_node_features = dataset[0].num_node_features

# Determine the number of graph-level features to be predicted
n_graph_features = len(dataset[0].y)

# Instantiate the models for nodes and edges
node_model = nGCNN(n_node_features, n_graph_features+1, dropout_node)
edge_model = eGCNN(n_node_features, n_graph_features+1, dropout_edge)

node_model.load_state_dict(torch.load(node_model_name, map_location=torch.device(device), weights_only=False))
edge_model.load_state_dict(torch.load(edge_model_name, map_location=torch.device(device), weights_only=False))
node_model.eval()
edge_model.eval()

# Allow data parallelization among multi-GPU
node_model= nn.DataParallel(node_model)
edge_model= nn.DataParallel(edge_model)

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
DataParallel(
  (module): nGCNN(
    (conv1): GraphConv(6, 256)
    (conv2): GraphConv(256, 512)
    (conv3): GraphConv(512, 256)
    (conv4): GraphConv(256, 4)
    (norm1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

Edge GCNN:
DataParallel(
  (module): eGCNN(
    (linear1): Linear(in_features=7, out_features=128, bias=True)
    (linear2): Linear(in_features=128, out_features=256, bias=True)
    (linear3): Linear(in_features=256, out_features=64, bias=True)
    (linear4): Linear(in_features=64, out_features=1, bias=True)
    (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


# Generating new cystals

In [31]:
# Predicting loop
diffused_dataset = []
with torch.no_grad():
    for idx in range(N_predictions):
        # Get random number of nodes
        n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        while n_nodes < 0:  # Make sure n_nodes is a positive integer from the normal distribution
            n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        
        # Get random graph, acting as diffused
        diffused_graph = get_random_graph(n_nodes, n_node_features)

        # Make room for n_graph_features and t_steps in the dataset
        diffused_graph = add_features_to_graph(diffused_graph,
                                               torch.tensor([target_tensor, 0], device=device))
        
        diffused_dataset.append(diffused_graph)
            
    # Generate batch objects
    diff_batch = Batch.from_data_list(diffused_dataset)
    
    # Move data to device
    diff_batch = diff_batch.to(device)
    
    # Denoise batch of diffused graphs
    predicted_dataset = denoise(diff_batch, n_t_steps, node_model, edge_model, alpha_decay=alpha_decay, sigma=model_parameters['sigma'])

# From batch object to list
predicted_dataset = predicted_dataset.to_data_list()

# Revert stardadization
denoised_graphs = revert_standardize_dataset(predicted_dataset, dataset_parameters)
denoised_graphs

RuntimeError: The size of tensor a (6) must match the size of tensor b (4) at non-singleton dimension 1

In [11]:
temp = denoised_graphs[0]

nodes = temp.x
edges = temp.edge_index.detach().cpu().numpy().T
weights = temp.edge_attr.detach().cpu().numpy()

In [12]:
nodes, weights

(tensor([[ 71.9272,  21.0472,   2.5597,   9.5406],
         [104.1651,  31.0890,   2.6542,   9.3892],
         [116.9652,  35.3267,   2.4768,  10.1779],
         [117.9008,  52.7503,   2.7047,  10.6533],
         [109.6857,  52.4652,   2.6629,  10.6234],
         [105.8927,  44.1827,   2.6604,  10.0779],
         [127.7625,  54.5869,   2.5684,  10.1005],
         [101.0349,  40.1833,   2.5784,   9.6555],
         [ 91.9999,  37.2730,   2.4036,   9.9699],
         [116.0556,  50.2209,   2.5086,  10.1145],
         [122.6762,  51.0751,   2.6933,  10.5572],
         [134.8280,  54.1657,   2.6574,  10.1510],
         [121.8354,  40.7826,   2.6859,  10.3472],
         [ 99.7271,  44.1657,   2.3131,   9.3684],
         [ 92.6044,  44.8905,   2.6500,  10.3383],
         [213.7116,  64.5890,   2.6510,   8.8843],
         [ 71.8913,  41.0737,   2.4813,   9.8112],
         [190.2209,  62.8202,   2.3617,   9.5126],
         [101.2071,  36.3617,   2.5780,   9.7299],
         [146.4583,  50.7582,  

In [13]:
import numpy as np
import torch

from scipy.optimize       import minimize
from libraries.graph      import graph_POSCAR_encoding, find_closest_key, composition_concentration_from_keys
from torch_geometric.data import Data
from pymatgen.core        import Structure


is_molecule = False

## Molecules

In [14]:
if is_molecule:
    # Initial guess for the positions
    solution = np.random.rand(len(nodes) * 3)*10  # Initialize all points at origin, 1D array
    #solution = coordinates.reshape(-1, 1).ravel()
    
    # Function to calculate the squared difference between distances and weights
    def objective(solution_attempt, edges, weights):
        positions = solution_attempt.reshape(-1, 3)  # Reshape to 2D array
        errors = 0
        for edge, weight in zip(edges, weights):
            p1 = positions[edge[0]]
            p2 = positions[edge[1]]
            distance = np.linalg.norm(p2 - p1)
            errors += np.power(distance - weight, 2)
        print(errors)
        return errors
    
    def worst_identification(edges, attributes, solution_attempt):
        positions = solution_attempt.reshape(-1, 3)  # Reshape to 2D array
    
        particle_errors = []
        for particle in np.unique(edges):
            # Get those edge indexes where particle has a connection
            particle_connections = np.where((edges[:, 0] == particle) | (edges[:, 1] == particle))
    
            particle_error = 0
            for idx in particle_connections[0]:
                # Load indexes in edge
                edge = edges[idx]
    
                # Load expected attribute
                p1 = positions[edge[0]]
                p2 = positions[edge[1]]
    
                # Load reference attribute
                weight = attributes[idx].item()
                
                # Compute error
                distance = np.linalg.norm(p2 - p1)

                # Append to trial errors for different atom images
                trial_error = np.power(distance - weight, 2)
    
                # Add error
                particle_error += trial_error
    
            # Average over the connection of the node
            particle_error /= len(particle_connections[0])
    
            # Append particle error
            particle_errors.append(particle_error)
    
        return np.argmax(particle_errors), np.max(particle_errors)

## Crystals

In [15]:
if not is_molecule:
    # Initial guess for the lattice parameters
    lattice_vectors = np.array([[10, 0,   0],
                                [0,   10, 0],
                                [0,   0,   10]])
    
    # Initial guess for the positions
    initial_positions = np.random.rand(len(nodes) * 3)  # Initialize all points at origin, 1D array
    #initial_positions = coordinates.reshape(-1, 1).ravel()
    solution = np.concatenate([lattice_vectors.ravel(), initial_positions])
    
    # Function to calculate the squared difference between distances and weights
    def objective(solution_attempt, edges, weights):
        solution_attempt = solution_attempt.reshape(-1, 3)  # Reshape to 2D array
        
        lattice_vectors = solution_attempt[:3]
        positions       = solution_attempt[3:]
        
        errors = 0
        for edge, weight in zip(edges, weights):
            p1 = positions[edge[0]]
            p2 = positions[edge[1]]
            
            trial_errors = [] 
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    for k in [-1, 0, 1]:
                        # i*lattice_vectors[0] + j*lattice_vectors[1] + k*lattice_vectors[2]
                        ijk_lattice_vectors = np.sum([i, j, k] * lattice_vectors.T, axis=1)

                        # Compute error
                        distance = np.linalg.norm(p2 - p1 + ijk_lattice_vectors)

                        # Append to trial errors for differente atom images
                        trial_errors.append(np.power(distance - weight, 2))
            errors += np.min(trial_errors)
        #print(errors)
        return errors
    
    def worst_identification(edges, attributes, solution_attempt):
        solution_attempt = solution_attempt.reshape(-1, 3)  # Reshape to 2D array
    
        lattice_vectors = solution_attempt[:3]
        positions       = solution_attempt[3:]
    
        particle_errors = []
        for particle in np.unique(edges):
            # Get those edge indexes where particle has a connection
            particle_connections = np.where((edges[:, 0] == particle) | (edges[:, 1] == particle))
    
            particle_error = 0
            for idx in particle_connections[0]:
                # Load indexes in edge
                edge = edges[idx]
    
                # Load expected attribute
                p1 = positions[edge[0]]
                p2 = positions[edge[1]]
    
                # Load reference attribute
                weight = attributes[idx].item()
    
                trial_errors = []
                for i in [-1, 0, 1]:
                    for j in [-1, 0, 1]:
                        for k in [-1, 0, 1]:
                            # i*lattice_vectors[0] + j*lattice_vectors[1] + k*lattice_vectors[2]
                            ijk_lattice_vectors = np.sum([i, j, k] * lattice_vectors.T, axis=1)
    
                            # Compute error
                            distance = np.linalg.norm(p2 - p1 + ijk_lattice_vectors)
    
                            # Append to trial errors for different atom images
                            trial_errors.append(np.power(distance - weight, 2))
    
                # Add error
                particle_error += np.min(trial_errors)
    
            # Average over the connection of the node
            particle_error /= len(particle_connections[0])
    
            # Append particle error
            particle_errors.append(particle_error)
    
        return np.argmax(particle_errors), np.max(particle_errors)

In [20]:
error_threshold = 1e-5

for attempt in range(100):
    print()
    print(f'Attempt {attempt}')
    solution = minimize(objective, solution,
                        args=(edges, weights),
                        method='Powell')

    is_success       = solution.success
    solution_message = solution.message
    worst_particle, worst_error = worst_identification(edges, weights, solution.x)

    attempt_error = objective(solution.x, edges, weights)
    print(f'Total: {attempt_error} and local {worst_error} errors')

    if attempt_error < error_threshold:
        break

    solution = solution.x.reshape(-1, 3)  # Reshape to 2D array

    # Re-initialize that position
    solution[worst_particle+3] = np.random.rand(3)

    solution = solution.flatten()

# Check convergence status
if is_success:
    print('Converged to a solution.')
else:
    print(f'Failed to converge: {solution_message}')


Attempt 0
Total: 2.309938997110655 and local 0.001681339479037708 errors

Attempt 1
Total: 2.306763882561487 and local 0.0016847482113030656 errors

Attempt 2
Total: 2.2954953720193885 and local 0.0015095455800962466 errors

Attempt 3
Total: 2.294899020058665 and local 0.001505644310177865 errors

Attempt 4
Total: 2.295161097717207 and local 0.0015100120370680605 errors

Attempt 5
Total: 2.2971136478068925 and local 0.0014992801363405389 errors

Attempt 6
Total: 2.289032667962247 and local 0.0015065632586896236 errors

Attempt 7
Total: 2.2890327985574106 and local 0.001504973148908069 errors

Attempt 8
Total: 2.289088239771534 and local 0.001518590576754965 errors

Attempt 9
Total: 2.3340758185522983 and local 0.0015462482662238137 errors

Attempt 10
Total: 2.293514373688564 and local 0.0015288544798004891 errors

Attempt 11


KeyboardInterrupt: 

In [21]:
solution = solution.reshape(-1, 3)

## Molecules

In [22]:
if is_molecule:
    # Get the position of each atom in direct coordinates
    #direct_positions = graph_to_cartesian_positions(graph)
    #cartesian_positions = solution.x.reshape(-1, 3)*mw
    #cartesian_positions = solution.x.reshape(-1, 3)
    
    lattice_vectors     = np.array([[10,  0,   0],
                                    [0,   10,  0],
                                    [0,   0,   10]])
    cartesian_positions = solution

## Crystals

In [23]:
if not is_molecule:
    # Get the position of each atom in direct coordinates
    #direct_positions = graph_to_cartesian_positions(graph)
    #cartesian_positions = solution.x.reshape(-1, 3)*mw
    
    lattice_vectors     = solution[:3]
    cartesian_positions = solution[3:]

In [25]:
POSCAR_name = None

# Get name for the first line of the POSCAR
POSCAR_name = POSCAR_name or 'POSCAR from GenerativeModels'

# Clone the input graph to preserve the original structure
new_graph = temp.clone()

# Load and detach embeddings for the graph nodes
data_embeddings = new_graph.x.detach().cpu().numpy()

# Loading dictionary of available embeddings for atoms
available_embeddings = {}
with open('../MP/input/atomic_masses.dat', 'r') as atomic_masses_file:
    for line in atomic_masses_file:
        key, mass, charge, electronegativity, ionization_energy = line.split()

        # Check if all information is present
        if all(val != 'None' for val in (mass, charge, electronegativity, ionization_energy)):
            available_embeddings[key] = np.array([mass, charge, electronegativity, ionization_energy], dtype=float)

# Get most similar atoms for each graph node and create a list of keys
keys = [find_closest_key(available_embeddings, emb) for emb in data_embeddings]

# Get elements' composition, concentration, and positions
POSCAR_composition, POSCAR_concentration, POSCAR_positions = composition_concentration_from_keys(keys, cartesian_positions)

In [26]:
# Write file
with open('CONTCAR', 'w') as POSCAR_file:
    # Delete previous data in the file
    POSCAR_file.truncate()
    
    # Write POSCAR's name
    POSCAR_file.write(f'{POSCAR_name}\n')

    # Write scaling factor (assumed to be 1.0)
    POSCAR_file.write('1.0\n')

    # Write lattice parameters (assumed to be orthogonal)
    np.savetxt(POSCAR_file, lattice_vectors, delimiter=' ')

    # Write composition (each different species, previously sorted)
    np.savetxt(POSCAR_file, [POSCAR_composition], fmt='%s', delimiter=' ')

    # Write concentration (number of each of the previous elements)
    np.savetxt(POSCAR_file, [POSCAR_concentration], fmt='%d', delimiter=' ')

    # Write position in cartesian form
    POSCAR_file.write('Cartesian\n')
    np.savetxt(POSCAR_file, POSCAR_positions, delimiter=' ')